<a href="https://colab.research.google.com/github/mmikite/MI_hzxa37/blob/main/10_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Egy egyszerű chatbot készítése (NLTK használatával)

## Szükséges könyvtárak importálása

In [ ]:
import io
import random
import string # to process standard python strings
import warnings
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

## Az NLTK letöltése és telepítése
Az NLTK (Natural Language Toolkit) egy vezető platform az emberi nyelvi adatokkal dolgozó Python programok építéséhez. Könnyen használható interfészeket biztosít több mint 50 korpuszhoz és lexikai forráshoz, mint például a WordNet, valamint szövegfeldolgozó könyvtárak csomagját az osztályozáshoz, tokenizáláshoz, törzseléshez, címkézéshez, elemzéshez és szemantikai következtetéshez, valamint csomagolásokat ipari erősségű NLP-könyvtárakhoz.

In [ ]:
!pip install nltk

### NLTK csomagok telepítése

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('popular', quiet=True) # for downloading packages
nltk.download('punkt') # first-time use only
nltk.download('wordnet') # first-time use only

## A corpus beolvasása

Példánkban a chatbotok Wikipedia oldalát fogjuk használni korpuszként; az oldal tartalmát belemásoltuk a "chatbot.txt" nevű szöveges fájlba. (Bármilyen tetszőleges korpuszt használhat.)

In [ ]:
#lokális adatok esetén
#f=open('chatbot.txt','r',errors = 'ignore')
#raw=f.read()
#raw = raw.lower()# converts to lowercase

In [ ]:
#távoli adatok esetén
import urllib.request as urllib2

raw = urllib2.urlopen("http://web.uni-corvinus.hu/~fszabina/data/chatbot.txt").read().decode("latin-1")
raw = raw.lower()# converts to lowercase

A fő probléma a szöveges adatokkal az, hogy azok mind szöveges formátumúak (karakterláncok). A gépi tanulási algoritmusoknak azonban valamilyen numerikus jellemzővektorra van szükségük a feladat elvégzéséhez. Mielőtt azonban bármilyen NLP-projektbe belekezdenénk, a szöveget elő kell készítenünk, hogy ideális legyen a munkához. Az alapvető szöveg előfeldolgozás magában foglalja:

* A teljes szöveg átalakítása **nagybetűssé** vagy **kisbetűssé**, hogy az algoritmus ne kezelje ugyanazokat a szavakat különböző esetekben különbözőként.

* **Tokenization**: A tokenizálás csak a normál szöveges karakterláncok tokenek, azaz a kívánt szavak listájává történő átalakításának folyamatát írja le. A mondat tokenizáló használható a mondatok listájának megtalálására, a szó tokenizáló pedig a szavak listájának megtalálására a karakterláncokban.

_Az NLTK adatcsomag tartalmaz egy előre betanított Punk tokenizálót az angol nyelvhez._


* A **zajok** eltávolítása, azaz minden, ami nem egy szabványos szám vagy betű.
* A **Stop szavak** eltávolítása. Néha néhány rendkívül gyakori szót, amelyeknek látszólag kevés értékük van a felhasználói igényeknek megfelelő dokumentumok kiválasztásában, teljesen kizárunk a szókincsből.

## Tokenizálás

In [ ]:
import nltk
nltk.download('punkt_tab')

sent_tokens = nltk.sent_tokenize(raw)# converts to list of sentences
word_tokens = nltk.word_tokenize(raw)# converts to list of words

## Előfeldolgozás

Egy LemTokens nevű függvényt definiálunk, amely bemenetként elfogadja a tokeneket, és normalizált tokeneket ad vissza.

In [ ]:
lemmer = nltk.stem.WordNetLemmatizer()

#WordNet is a semantically-oriented dictionary of English included in NLTK.
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

## Üdvözlés - kulcsszó illesztés

Definiálunk egy függvényt a bot üdvözlésére, azaz ha a felhasználó bemenete egy üdvözlés, a bot egy üdvözlő választ ad vissza. Egyszerű kulcsszó illesztést fogunk használni.

In [ ]:
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]
def greeting(sentence):

    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

## Válasz elkészítése

### TF-IDF megközelítés
a szavak gyakoriságának átméretezése aszerint, hogy milyen gyakran fordulnak elő az összes dokumentumban, így az olyan gyakori szavak pontszámai, mint a "the", amelyek szintén gyakoriak az összes dokumentumban, büntetést kapnak. A pontozásnak ezt a megközelítését Term Frequency-Inverse Document Frequency, röviden TF-IDF-nek nevezik, ahol:

**Term Frequency: a szó gyakoriságának pontozása az aktuális dokumentumban.**

```
TF = (Number of times term t appears in a document)/(Number of terms in the document)
```

**Inverse Document Frequency: annak pontozása, hogy a szó mennyire ritka a dokumentumokban.**

```
IDF = 1+log(N/n), ahol, N a dokumentumok száma, n pedig azon dokumentumok száma, amelyekben a t kifejezés megjelent.
```
### Cosinus távolság (Cosine similarity)

A Tf-idf súly az információkeresésben és a szövegbányászatban gyakran használt súly. Ez a súly egy statisztikai mérőszám, amelyet annak értékelésére használnak, hogy egy szó mennyire fontos egy dokumentumban egy gyűjteményben vagy korpuszban.

```
Cosine távolság (d1, d2) =  Dot product(d1, d2) / ||d1|| * ||d2||
```
ahol d1,d2 két nem nulla vektor.



Ahhoz, hogy a robotunk válaszokat generáljon a bemeneti kérdésekre, a dokumentumhasonlóság fogalmát fogjuk használni. Definiálunk egy válasz függvényt, amely a felhasználó kijelentését egy vagy több ismert kulcsszóra keresi, és több lehetséges válasz közül egyet ad vissza. Ha nem talál a bemenetre illeszkedő kulcsszót, akkor a következő választ adja vissza:" Sajnálom! Nem értem Önt"

In [ ]:
def response(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    #A flatten() metódus egy többdimenziós NumPy tömböt egydimenziós tömbbé alakít.
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

Végül, a felhasználó bevitelétől függően betápláljuk azokat a sorokat, amelyeket a botunknak mondania kell a beszélgetés megkezdésekor és befejezésekor.

In [ ]:
flag=True
print("ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("ROBO: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("ROBO: "+greeting(user_response))
            else:
                print("ROBO: ", end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("ROBO: Bye! take care..")